I think for the rms reactor, the main loop that we will want to worry about is the one in EdgeAnalysis.jl, "identify_objects", line  676: 

```
for i = 1:length(edgespeciesrateratios)
    if @inbounds  edgespeciesrateratios[i] > maxedgespeciesrateratios[i]
        @inbounds maxedgespeciesrateratios[i] = edgespeciesrateratios[i]
    end
end
```

this line iterates through all of the edge species. 

fyi rms reactors are in the rmgpy/rmg/reactors.py module. 